In [ ]:
pip install azure-storage-blob
pip install pandas
pip install matplotlib

In [ ]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import matplotlib.pyplot as plt
import json

In [ ]:
account_name = 'sacps'
account_key = 'LtaJD/kCVNP/0MPb98xEjzvmZdeQ8zj6lZXT15GA//XDV7itlgY1FOfWcYEaC5ixnXwYixwhxs5Q+AStVQt95Q=='
container_name = 'sinkcontcps'

#create a client to interact with blob storage
connect_str = 'DefaultEndpointsProtocol=https;AccountName=' + account_name + ';AccountKey=' + account_key + ';EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

#use the client to connect to the container
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
json_data_list = []

for blob in container_client.list_blobs():
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob.name)
    blob_data = blob_client.download_blob()
    
    
    json_content = json.loads(blob_data.readall())
    json_data_list.append(json_content)

In [ ]:
df = pd.DataFrame(json_data_list)

In [ ]:
print(len(df.index))

In [ ]:
df

In [ ]:
print(df.columns)


In [ ]:
print(df.head())


In [ ]:
df['dt'] = pd.to_datetime(df['dt'], unit='s')

#
temperatures_k = [entry['temp'] for entry in df['main']]
temperatures_celsius = [temp - 273.15 for temp in temperatures_k]

#
plt.figure(figsize=(10, 6))
plt.plot(df['dt'], temperatures_celsius, marker='o', linestyle='-', color='b')
plt.title("Temperature Over Time (Celsius)")
plt.xlabel("Time")
plt.ylabel("Temperature (°C)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()